# FACET-II Linac de-splitting

In [2]:
from pytao import Tao
from pprint import pprint
import json


In [3]:
M = Tao('-init $FACET2_LATTICE/bmad/models/f2_elec/tao.init -noplot')

In [4]:
%%tao
python lat_list 1@0>>Q*|base real:ele.s

-------------------------
Tao> python lat_list 1@0>>Q*|base real:ele.s
-------------------------
Tao> 


In [5]:
SLIST = M.cmd_real('python lat_list -track_only 1@0>>*|model real:ele.s')
LLIST = M.cmd_real('python lat_list -track_only  1@0>>*|model real:ele.l')
NAMES = M.cmd('python lat_list -track_only 1@0>>*|model ele.name')
IXLIST = [i for i in range(len(NAMES))]

ix_of = {}
for ix, name in enumerate(NAMES):
    if name in ix_of:
        pass
#        print("err", name)
    else: 
        ix_of[name] = ix

for s, l, n in zip(SLIST[0:20], LLIST[0:20], NAMES[0:20]):
    print (n, l, s)

BEGINNING 0.0 0.0
DL00 -0.8690480010000002 -0.8690480010000002
LOADLOCKF 0.8690480010000002 0.0
BEGINJ 0.0 0.0
SOL10111 0.0 0.0
CATHODEF 0.0 0.0
DL01A 0.0972947 0.0972947
SOL10121#1 0.1 0.1972947
YC10122 0.0 0.1972947
XC10121 0.0 0.1972947
SQ10122 0.0 0.1972947
CQ10121 0.0 0.1972947
SOL10121#2 0.1 0.2972947
DL01B 0.0785101 0.37580480000000005
VV10155 0.0 0.37580480000000005
DL01C 0.1160862 0.4918910000000001
MIR10181 0.0 0.4918910000000001
DL01D 0.0811358 0.5730268000000001
VV10215 0.0 0.5730268000000001
DL01E 0.1263224 0.6993492000000001


In [6]:
# Search for split eles

def find_split_eles(slist, llist, names):
    """
    Searches for split elements
    
    """

    split_eles = {}
    basename = 'xxx'
    split_eles[basename] = {'stubs':[], 'splits':[], 'offsets':[]}
    stubs  = split_eles[basename]['stubs']
    splits = split_eles[basename]['splits']
    offsets = split_eles[basename]['offsets']
    s0 = 0
    for s, l, n in zip(SLIST, LLIST, NAMES):
        
        if l == 0:
            splits.append(n)
            offsets.append(s - s0)
        # Thick ele
        elif n.startswith(basename) and len(n)==len(basename)+1:
            # this is a true split ele     
            stubs.append(n)
            split_eles[basename]['L'] += l
        else:
            # New thick ele
            
            # Removing unwanted eles
            if basename == 'K21_3B':
                # Special case. Leave these
                print(basename, stubs, splits)
                pass
            elif len(stubs) == 1 or len(splits) == 0:
                split_eles.pop(basename)
            # Or if this is obviously a superimpose element
            elif basename.endswith('#'):
                split_eles.pop(basename)
                
            # Try a basename which excludes the final character
            basename = n[:-1]
            split_eles[basename] = {'stubs':[], 'splits':[], 'offsets':[]}
            split_eles[basename]['L'] = l
            s0 = s -l 
            stubs  = split_eles[basename]['stubs']
            splits = split_eles[basename]['splits']
            offsets = split_eles[basename]['offsets']
            stubs.append(n) # add this 
    
        #print (n, l, s, ix)
        
    split_eles.pop('xxx')        
    
    return split_eles

SPLIT_ELES = find_split_eles(SLIST, LLIST, NAMES)
    
pprint(SPLIT_ELES)

{'D11OD': {'L': 0.2884622642981027,
           'offsets': [0.1312309357279986],
           'splits': ['CE11345'],
           'stubs': ['D11OD1', 'D11OD2']},
 'D199': {'L': 4.879374683414004,
          'offsets': [4.570883999999978,
                      4.879374683413971,
                      4.879374683413971,
                      4.879374683413971],
          'splits': ['IM1988', 'LI19TERM', 'ENDL3F_2', 'BEGBC20'],
          'stubs': ['D199C', 'D199D']},
 'D199A': {'L': 1.1216,
           'offsets': [0.8016000000000076, 1.1216000000000577],
           'splits': ['XC19900', 'YC19900'],
           'stubs': ['D199A1', 'D199A2']},
 'D1ET1': {'L': 3.8659737036155786,
           'offsets': [2.8210129999999936, 3.865973703615623],
           'splits': ['XC1996', 'IM2040'],
           'stubs': ['D1ET1A', 'D1ET1B']},
 'D1ET2': {'L': 3.8684887036155784,
           'offsets': [2.452947999999992],
           'splits': ['XC2460'],
           'stubs': ['D1ET2A', 'D1ET2B']},
 'D210B': {'L': 0.2,


# Desplit klystrons

In [7]:
def lines_from_splits(name, split_ele_dict):
    offsets = split_ele_dict['offsets']
    splits = split_ele_dict['splits']
    stubs = split_ele_dict['stubs']
    L = round(split_ele_dict['L'], 9)
    
    

    
    lines = ['!---------------------', f'! {name} LCAVITY']

    
    ###     lines.append(f'{name}: {stubs[0]}, L = {L}')
    # These don't yet have full elements
    
    # Mark is putting in _full lines for these, so shoun't need special logic
    if name in []: #['L0AF', 'L0BF']:
        #lines.append(f'{name}: {stubs[0]}, L = {L}')
        
        voltage = '+'.join([s.strip()+'[voltage]' for s in stubs])
        
        lines.append(f'{name}: {stubs[0]}, L = {L}, voltage = {voltage}')        
        

    lines.append(f'{name}_full: line = ({name})')
    
    if len(splits) == 0:
        lines.append('! does not contain splitting elements. Consider reforming')
    else:
        lines.append('! contains zero length elements:')
    
    for n, o in zip(splits, offsets):
        o = round(o, 9)
        # This is at the end of the ele. Skip markers that shouldn't be included
        if o == L:
            if n.startswith('BEG'):
                continue
            elif n.startswith('END'):
                continue                
        
        lines.append(f'    {n}[superimpose] = T')
        lines.append(f'    {n}[ref] = {name}')
        lines.append(f'    {n}[ref_origin] = beginning')
        lines.append(f'    {n}[offset] = {o}')
    lines.append('\n')
    
    return lines



CU_LINAC_REPLACEMENTS = {}

for name, ele in SPLIT_ELES.items():
    if any([name.startswith(x) for x in ['K', 'L0A', 'L0B', 'L1X'] ]) :
        
        # Strip off ___ from some of these
        name = name.strip('_')
        
        CU_LINAC_REPLACEMENTS[name] = '\n'.join(lines_from_splits(name, ele))
    
    
    
    #line = '\n'.join(lines_from_splits(name, ele))
    
with open('facet2_linac_replacements.json', 'w') as outfile:
    json.dump(CU_LINAC_REPLACEMENTS, outfile, ensure_ascii=True, indent='  ')

In [8]:
pprint(CU_LINAC_REPLACEMENTS)

{'K11_2A': '!---------------------\n'
           '! K11_2A LCAVITY\n'
           'K11_2A_full: line = (K11_2A)\n'
           '! contains zero length elements:\n'
           '    XC11202[superimpose] = T\n'
           '    XC11202[ref] = K11_2A\n'
           '    XC11202[ref_origin] = beginning\n'
           '    XC11202[offset] = 0.3256\n'
           '    YC11203[superimpose] = T\n'
           '    YC11203[ref] = K11_2A\n'
           '    YC11203[ref_origin] = beginning\n'
           '    YC11203[offset] = 0.6126\n'
           '\n',
 'K11_4A': '!---------------------\n'
           '! K11_4A LCAVITY\n'
           'K11_4A_full: line = (K11_4A)\n'
           '! contains zero length elements:\n'
           '    XC11402[superimpose] = T\n'
           '    XC11402[ref] = K11_4A\n'
           '    XC11402[ref_origin] = beginning\n'
           '    XC11402[offset] = 0.3268\n'
           '    YC11403[superimpose] = T\n'
           '    YC11403[ref] = K11_4A\n'
           '    YC11403[ref_origin

In [9]:
!mv facet2_linac_replacements.json good_facet2_linac_replacements.json